In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv("../nt_data/logs/attendance-2021-02-01.csv", names=['id', 'datetime', 'distance', 'emotion'])

In [3]:
df.head()

,id,datetime,distance,emotion
1007,Umme Hani,2021-02-01 11:26:06.528505,0.739909,neutral
1007,Umme Hani,2021-02-01 11:26:07.252654,0.748936,neutral
1007,Umme Hani,2021-02-01 11:26:07.461598,0.745613,neutral
1007,Umme Hani,2021-02-01 11:26:07.575653,0.749254,neutral
1007,Umme Hani,2021-02-01 11:26:08.249940,0.747589,neutral


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473 entries, 0 to 472
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        473 non-null    int64  
 1   datetime  473 non-null    object 
 2   distance  473 non-null    float64
 3   emotion   473 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 14.9+ KB


# ToDo
* Daily Report
** Employee Id
** First Punch
** Last Punch
** Spent hours
** Percentile
** Emotion

In [4]:
# cleaning the dataset
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
# converting 'datetime' to pandas datetime series
df['datetime'] = pd.to_datetime(df['datetime'])
df.sort_values(by='datetime', inplace=True)
df['date'] = df['datetime'].apply(lambda x: x.date())
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 311 entries, 1007 to 1018
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        311 non-null    object        
 1   datetime  311 non-null    datetime64[ns]
 2   distance  311 non-null    float64       
 3   emotion   311 non-null    object        
 4   date      311 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 14.6+ KB


In [5]:
df.head()

,id,datetime,distance,emotion,date
1007,Umme Hani,2021-02-01 11:26:06.528505,0.739909,neutral,2021-02-01
1007,Umme Hani,2021-02-01 11:26:07.252654,0.748936,neutral,2021-02-01
1007,Umme Hani,2021-02-01 11:26:07.461598,0.745613,neutral,2021-02-01
1007,Umme Hani,2021-02-01 11:26:07.575653,0.749254,neutral,2021-02-01
1007,Umme Hani,2021-02-01 11:26:08.249940,0.747589,neutral,2021-02-01


In [9]:
gdf = df.groupby('id')

In [10]:
report = gdf.first()

In [11]:
report.rename(columns={'datetime': 'first_punch'}, inplace=True)

In [12]:
report['last_punch'] =gdf.last()['datetime']

In [13]:
report['timedelta'] = report['last_punch'] - report['first_punch']

In [14]:
report['spent'] = report['timedelta'].apply(lambda x: (datetime.fromisoformat("1990-01-01") + x).strftime("%H:%M:%S"))

In [15]:
report['percentile'] = report['timedelta'].apply(lambda x: int(((x.total_seconds()) / (8*3600)) * 100))

In [16]:
report = report.drop(['distance', 'timedelta'], axis=1)

In [17]:
def get_remarks(time):
    time = time.to_pydatetime().time()
    remark = None
    # predefined time constraints
    in_time = datetime.strptime('11:30:59', '%H:%M:%S').time()
    late_start = datetime.strptime('11:31:00', '%H:%M:%S').time()
    late_end = datetime.strptime('13:00:59', '%H:%M:%S').time()
    half_start = datetime.strptime('13:01:00', '%H:%M:%S').time()
    half_end = datetime.strptime('15:30:59', '%H:%M:%S').time()
    late_half_start = datetime.strptime('15:31:00', '%H:%M:%S').time()
    late_half_end = datetime.strptime('16:30:59', '%H:%M:%S').time()
    absent_time = datetime.strptime('16:31:00', '%H:%M:%S').time()

    # in time
    if time > absent_time:
        remark = 'Absent'
    if late_half_start <= time <= late_half_end:
        remark = 'Late Half'
    if half_start <= time <= half_end:
        remark = 'Half Day'
    if late_start <= time <= late_end:
        remark = 'Late'
    if time <= in_time:
        remark = 'Normal'

    return remark

def calculate_deficiency(spent):
    actual_hour = datetime.strptime('08:00:00', '%H:%M:%S')
    spent = datetime.strptime(spent, '%H:%M:%S')
    deficit = actual_hour - spent

    deficit_td = (datetime.fromisoformat("1990-01-01") + deficit).strftime("%H:%M:%S")\
                 if deficit.days >= 0 else (datetime.fromisoformat("1990-01-01") + timedelta(0)).strftime("%H:%M:%S")
    return deficit_td

In [18]:
report['remarks'] = report['first_punch'].apply(lambda x: get_remarks(x))

In [19]:
report['first_punch'] = report['first_punch'].apply(lambda x: x.strftime("%H:%M:%S"))
report['last_punch'] = report['last_punch'].apply(lambda x: x.strftime("%H:%M:%S"))
report['deficiency'] = report['spent'].apply(lambda x: calculate_deficiency(x))

In [20]:
report

,first_punch,emotion,date,last_punch,spent,percentile,remarks,deficiency
id,,,,,,,,
102,11:27:35,neutral,2021-01-19,19:33:55,08:06:20,101,Normal,00:00:00
105,11:21:54,neutral,2021-01-19,13:21:56,02:00:01,25,Normal,05:59:59
1001,11:03:07,angry,2021-01-19,11:03:10,00:00:03,0,Normal,07:59:57
1002,11:09:33,neutral,2021-01-19,19:25:42,08:16:08,103,Normal,00:00:00
1007,11:27:18,sad,2021-01-19,19:34:03,08:06:44,101,Normal,00:00:00
1011,11:37:54,fear,2021-01-19,15:59:21,04:21:26,54,Late,03:38:34
1014,11:47:27,neutral,2021-01-19,19:31:12,07:43:45,96,Late,00:16:15
1015,11:37:54,fear,2021-01-19,17:59:31,06:21:36,79,Late,01:38:24
1017,11:58:27,fear,2021-01-19,11:58:31,00:00:04,0,Late,07:59:56


In [21]:
report.reset_index(inplace=True)

In [22]:
report = report[['id', 'date', 'first_punch', 'last_punch', 'spent',  'deficiency', 'percentile', 'remarks']]

In [23]:
report.reset_index(drop=True)

,id,date,first_punch,last_punch,spent,deficiency,percentile,remarks
0,102,2021-01-19,11:27:35,19:33:55,08:06:20,00:00:00,101,Normal
1,105,2021-01-19,11:21:54,13:21:56,02:00:01,05:59:59,25,Normal
2,1001,2021-01-19,11:03:07,11:03:10,00:00:03,07:59:57,0,Normal
3,1002,2021-01-19,11:09:33,19:25:42,08:16:08,00:00:00,103,Normal
4,1007,2021-01-19,11:27:18,19:34:03,08:06:44,00:00:00,101,Normal
5,1011,2021-01-19,11:37:54,15:59:21,04:21:26,03:38:34,54,Late
6,1014,2021-01-19,11:47:27,19:31:12,07:43:45,00:16:15,96,Late
7,1015,2021-01-19,11:37:54,17:59:31,06:21:36,01:38:24,79,Late
8,1017,2021-01-19,11:58:27,11:58:31,00:00:04,07:59:56,0,Late
9,1018,2021-01-19,11:37:48,13:21:20,01:43:31,06:16:29,21,Late


In [24]:
report.to_csv('../app/media/report.csv')

In [28]:
di = dict(report.iterrows())

In [39]:
data = [dict(x) for x in di.values()]